In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Sentiment Analysis

The objective of sentiment analysis is to quantify the attitudes expressed in a body of text.  In the simplest case, we want to determine if the sentiment of text is positive or negative.  This is a text classification problem, so the methods that we discussed before all apply.  However, there are some nuances that we should keep in mind:

* **Number of Classes:** We'll stick to binary classification in this notebook for simplicity, but it's common to add a third class for neutral statements that don't express positive or negative sentiment.  More ambitious classifiers might include additional classes to encode intensity of sentiment or target specific emotions.   
* **Context Dependence:** Use of language varies significantly based on context.  For example, the word *cheesy* is more likely to be negative in the context of a movie review, and more likely to be positive if the topic is pizza.  So it's best to use training data that closely resembles your intended use case.   
* **Imperfect Classification:** Sentiment is subjective, so even human coders will not label sentences with 100% consistency.  In other words, this is a hard problem, and we should expect that any classifier we train will make mistakes some of the time.  We can identify trends in sentiment across large data sets, but we cannot reliably classify the sentiment of individual statements.            

## Bag of Words Model

In this notebook we'll work with a movie review data set consisting of 1000 positive reviews and 1000 negative reviews.  We'll start by downloading the data set.

In [ ]:
import nltk                            # nltk is another nlp library
nltk.download("movie_reviews")         # try nltk.download() with no arguments to see additional options
from nltk.corpus import movie_reviews

In [ ]:
files = movie_reviews.fileids()        
files[:3]+files[-3:]  # Example filenames                   

In [ ]:
print(movie_reviews.raw(files[-2])[:400]+"...")  # Example of a positive review

To prepare the data for machine learning, we shuffle the positive and negative reviews together, perform a train-test split, and extrapolate training labels from the file names.

In [ ]:
import random
random.seed(200) # For consistency 
files = movie_reviews.fileids() # For consistency if the cell is run again 

from random import shuffle
shuffle(files)  # Randomizes the order of the files, changing the list in place

train = files[:1600] 
test = files[1600:]

X_train = [movie_reviews.raw(movie) for movie in train]
X_test = [movie_reviews.raw(movie) for movie in test]

def label(movie):
    return "Positive" if movie[:3]=="pos" else "Negative"

y_train = [label(movie) for movie in train]
y_test = [label(movie) for movie in test]

We can make a simple and effective pipeline just by combining a vectorizer with a Naive Bayes classifier. More details on this type of classifier can be found in the Naive Bayes notebook.  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

est = Pipeline([('vectorizer', TfidfVectorizer()),
                ('classifier', MultinomialNB())])

est.fit(X_train,y_train)
est.score(X_test,y_test) # Returns the accuracy of the model on the test set

For a better model, let's use some of the tools we discussed before in the __[Natural Language Processing notebook](AM_Natural_Language_Processing.ipynb)__.  We'll include bigrams, exclude stopwords, and use lemmatization.  Note that we've disabled a number of steps in the `spaCy` language processing pipeline.  By default, `spaCy` does a substantial amount of parsing, part-of-speech tagging, and other processing that is not necessary if all we want to do is get word lemmas.  Disabling these components of the `spaCy` pipeline results in substantially faster tokenization.

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en', disable=['parser','tagger','ner','textcat'])

def tokenize_lemma(text):
    return [w.lemma_ for w in nlp(text)]

stop_words_lemma = set(w.lemma_ for w in nlp(' '.join(STOP_WORDS)))

est = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1,2),
                                               stop_words=stop_words_lemma,
                                               tokenizer=tokenize_lemma)),
                ('classifier', MultinomialNB())])

est.fit(X_train,y_train) # This may take 2 or 3 minutes to run 
est.score(X_test,y_test) 

We can also use a more sophisticated machine learning algorithm in place of Naive Bayes.  Two good candidates are Logistic Regression and Linear SVM.

However, these models may train slowly on NLP data sets because of the large number of features.  One solution is to train them using stochastic gradient descent (which only considers one sample at each training step) instead of gradient descent (which considers the entire sample at each step).  Sklearn's `SGDClassifier` can be used to implement a variety of linear models using stochastic gradient descent.

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_pipe = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1,2),
                                               stop_words=STOP_WORDS)), # lemmatization removed to improve runtime
                     ('classifier', SGDClassifier(max_iter=5))])

It may not be obvious, but we've implicitly made some choices in defining the pipeline above:

* `SGDClassifier` implements Linear SVM by default, but we want to try Logistic Regression as well.
* `SGDClassifier` tries to combat overfitting using regularization, a penalty which increases with model complexity.  The strength of this penalty is controlled by a hyperparameter, alpha.  alpha defaults to 0.0001, but this isn't necessarily the optimal value.

Instead of guessing at the best choices, we can systematically search over a grid of hyperparameter values and select the model with the best cross-validation score.  We can use the `GridSearchCV` function in Scikit-Learn to do this.   

In [ ]:
from sklearn.model_selection import GridSearchCV          

parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2)], 
              'classifier__alpha': (0.001, 0.0001, 0.00001),
              'classifier__loss': ('log', 'hinge'), # log = Logistic Regression, hinge = Linear SVM
}

grid = GridSearchCV(sgd_pipe, parameters, cv=5)
grid.fit(X_train,y_train) # This may take 2 or 3 minutes to run 

est = grid.best_estimator_  #  Let's have a closer look at the best parameters...
est.score(X_test,y_test) 

We can see the best parameters that `GridSearchCV` found using the `named_steps` attribute, which allows us to
see those parameter.  

In [ ]:
est.named_steps['classifier']

## Interpreting the Model

The models that we trained in this notebook have a high degree of interpretability.  In contrast to other machine learning models, which are effectively black boxes, we can say a lot about how predictions are being made:

* We can infer feature importance from the coefficients of the model.  In other words, we can identify the words that most strongly signal positive and negative sentiment.
* We can look at the probability values associated with model predictions to judge the model's confidence in those predictions. 

For example, let's look at the more basic Naive Bayes model.

In [ ]:
est = Pipeline([('vectorizer', TfidfVectorizer(min_df=50)), # min_df=50 because we're not interested in uncommon words
                ('classifier', MultinomialNB())])
est.fit(X_train,y_train)

vocab = est.get_params()['vectorizer'].vocabulary_ # A dictionary of (word, value) pairs
                                                   # vocab['word'] is the index in the coefficient array corresponding to 'word'
    
coeff_pos = est.get_params()['classifier'].feature_log_prob_[1] 
coeff_neg = est.get_params()['classifier'].feature_log_prob_[0]

The coefficients from the model tell us how likely each word is to occur in reviews of each class:

$$\mbox{coeff_pos[vocab['word']]} = \log \bigl( P(\mbox{ Review contains word }|\mbox{ Review is positive })\bigr)$$

$$\mbox{coeff_neg[vocab['word']]} = \log \bigl( P(\mbox{ Review contains word }|\mbox{ Review is negative })\bigr)$$

One way to measure the "polarity" of a word is just to look at the difference between these two coefficients.  Let's identify the most positively and negatively charged words in our corpus.

In [ ]:
from numpy import argsort

polarity = coeff_pos - coeff_neg
indices = argsort(polarity) # indices of the polarity list, sorted from least to greatest

print("Positive Words \n-----")
for word in vocab:
    if vocab[word] in indices[-10:]:
        print(word)
        
print("\nNegative Words \n-----")
for word in vocab:
    if vocab[word] in indices[:10]:
        print(word)        

One simple way to visualize the sentiment of text is to color polarized words.  Below we'll write a function that both colorizes the text of reviews from the test set and summarizes the output from our model.  

In [ ]:
cutoff = 250

best_N = indices[-cutoff:] # Indices of the best/worst 250 words
worst_N = indices[:cutoff]

best_N = [word for (word,index) in vocab.items() if index in best_N] # Lists of the words themselves
worst_N = [word for (word,index) in vocab.items() if index in worst_N]

In [ ]:
from colorama import Fore, Style
import re

def colorize(text): # Colors the most polarized words             
    for word in best_N:
        text = re.sub(r"\b"+word+r"\b",Fore.GREEN+Style.BRIGHT+word+Fore.RESET+Style.RESET_ALL,text)  
    for word in worst_N:
        text = re.sub(r"\b"+word+r"\b",Fore.RED+Style.BRIGHT+word+Fore.RESET+Style.RESET_ALL,text)
    return text

def summary(n, short=True): # Summarizes model performance for the specified review in the test set
    text = X_test[n]
    if short:
        text = text[:500]+"..."
    print("Review", "#"+str(n))
    print("Actual Class:", y_test[n])
    print("Model Prediction:", est.predict([text])[0])
    print("Confidence:", max(est.predict_proba([text])[0][0], est.predict_proba([text])[0][1])) # Probability
    print("\n" + colorize(text) + "\n")

In [ ]:
summary(14)  # Some examples
summary(143)
summary(192)

## Grammar and Other Tools

The bag of words approach has its limitations.  It's not hard to imagine sentences that will confuse our models.  For example:

> "I was expecting an uninspired remake, but this movie wasn't bad at all.  There was no lame dialogue, and the plot twist wasn't terrible, so I'd actually say that the original was worse."

In order to parse sentences like these, we need some notion of grammar.  A simple grammar is something we can write by hand.  It's just a set of rules that say how the parts of a sentence can fit together.  

In [ ]:
simple_grammar = nltk.CFG.fromstring("""
... S -> NP VP
... NP -> DT NN | DT JJ NN
... VP -> VBD PP
... PP -> IN NP
... DT -> 'The' | 'the'
... NN -> 'fox' | 'dog'
... VBD -> 'jumped'
... IN -> 'over'
... JJ -> 'lazy'
... NN -> 'dog'
... """)

In [ ]:
parser = nltk.ChartParser(simple_grammar)
sentence = "The fox jumped over the lazy dog"
trees = parser.parse(sentence.split(" ")) # The input is an ordered list of words and the output is a generator
                                          # Depending on the grammar, there may be different ways to parse the same sentence. 
for tree in trees:
    tree.pretty_print()

Creating and using grammars for practical text analysis is more challenging:

* Parsing will fail whenever it encounters words or structures not included in the grammar, so we're obligated to include a large number of each.
* We're likely to get ambiguous results (many alternative parse trees) when using grammars with many rules, especially when we try to parse long sentences.  
* Ambiguity can be reduced using probabilistic parsing (based on a weighted grammar), but this further increases the complexity of the task. 

The simplest approach is to avoid these issues by using tools that already exist.  One such tool is Stanford's [CoreNLP](https://stanfordnlp.github.io/CoreNLP/).  CoreNLP is written in Java, so it requires a bit of setup, but it has Python wrappers like [`py-corenlp`](https://github.com/smilli/py-corenlp), and its parsing capabilities are more comprehensive than pure Python alternatives.  CoreNLP also includes a sentiment analysis tool that uses [deep learning](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf).

Another option is `spaCy`, which has built in parsing capabilities.  However, `spaCy` only keeps track of dependencies between words, so we won't get all of the higher level information that we had before.  Since structures like NP and VP aren't included, we get a shallower tree.

In [ ]:
from nltk import Tree

nlp = spacy.load('en', disable=['tagger','ner','textcat'])
def to_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree("-".join([node.text, node.tag_]), [to_tree(child) for child in node.children])
    else:
        return "-".join([node.text, node.tag_])

sentence = next(nlp(u"The fox jumped over the lazy dog").sents)    
    
to_tree(sentence.root).pretty_print()

### Final Note

While the appeal of more sophisticated methods is obvious, we should not feel obligated to use them in every circumstance.  At the sentence level, even state-of-the-art methods have misclassification rates over 10%, so in many circumstances they don't offer a practical advantage over bag of words.  We should use grammars and parsing only if the complexity of the problem makes it necessary.  For the example, in this notebook it was sufficient to identify polarized words in a general way because the topics of the reviews are known.  However, if we were trying to do topic identification and sentiment analysis simultaneously, then we would need a way to link polarized words to their targets.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*